In [19]:
import numpy as np
from PIL import Image
import PIL
import os
import shutil
import imageio
import cv2
import random

In [26]:
#HYPER PARAMETERS
#Program handles process of creating pictures
#It imports injection(marker, named "injection.png"), bakcground photo(from src_dir), combines them by injecting marker in random position of background
#saves it later to dst_dir
def main():
    src_dir = './Backgroundpics/images'
    dst_dir = './Train'
    file_ext = '.jpeg'
    size_of_dataset = 100
    make_data(src_dir, dst_dir, limit = size_of_dataset, file_ext = file_ext)

In [27]:
def cvpaste(img, imgback, x, y, angle, scale):  
        # x and y are the distance from the center of the background image 
        r = img.shape[0]
        c = img.shape[1]
        rb = imgback.shape[0]
        cb = imgback.shape[1]
        hrb=round(rb/2)
        hcb=round(cb/2)
        hr=round(r/2)
        hc=round(c/2)

        # Copy the forward image and move to the center of the background image
        imgrot = np.zeros((rb,cb,3),np.uint8)
        imgrot[hrb-hr:hrb+hr,hcb-hc:hcb+hc,:] = img[:hr*2,:hc*2,:]

        # Rotation and scaling
        M = cv2.getRotationMatrix2D((hcb,hrb),angle,scale)
        imgrot = cv2.warpAffine(imgrot,M,(cb,rb))
        # Translation
        M = np.float32([[1,0,x],[0,1,y]])
        imgrot = cv2.warpAffine(imgrot,M,(cb,rb))

        # Makeing mask
        imggray = cv2.cvtColor(imgrot,cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(imggray, 10, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)

        # Now black-out the area of the forward image in the background image
        img1_bg = cv2.bitwise_and(imgback,imgback,mask = mask_inv)

        # Take only region of the forward image.
        img2_fg = cv2.bitwise_and(imgrot,imgrot,mask = mask)

        # Paste the forward image on the background image
        imgpaste = cv2.add(img1_bg,img2_fg)

        return imgpaste


In [28]:
#Proceed with injetion on background photo
def inject_photo(background, name, dst_dir, injected_photo='injection.png'):
    """Function makes injection into 'Background' from ' injected_photo', renames it and places it in dst_dir """

    imgback = cv2.imread(background,-1)
    img = cv2.imread(injected_photo,-1)
    rows,cols,channels = imgback.shape
    x=(random.uniform(-cols/4,cols/4)) # x distancec for x axis from center 
    y=(random.uniform(-rows/4,cols/4)) # y y distance from center
    angle=(random.randint(0, 360)) # decide angle from 0 to 360
    scale=(random.uniform(0.5,2))    # chose the sholipe logo sacale 

    imgpaste = cvpaste(img, imgback, x, y, angle, scale)
    name = os.path.join(dst_dir, name)
    cv2.imwrite(name,imgpaste)

In [29]:
def copy_rename(background,name, dst_dir):
    """Function converts background picture to 4 channel picture in dst_dir with new name"""
    #import
    img = Image.open(background)
    #convert to 4 channel
    new_img = img.convert('RGB')
    #get new name
    new_name = os.path.join(dst_dir,name)
    #Save
    new_img.save(new_name)

In [30]:
def make_data(src_dir, dst_dir, limit = 500, file_ext='.png', img_size = 256, num_channels = 3):
#Params
    n = 0;
    num_pics = len(os.listdir(src_dir))
    label = np.zeros(shape=(limit,2))
    mark = np.array([[1,0]])
    print("creating {} samples".format(limit))
#Creation of data
#Needs photos with 
#in first loop creates injected photos, in second copies photos
#Mark with 1 in first coulumn tells that injection is present, in second that it is absent(one-hot code)
    for i in range(1,3):
        if (i==2): mark = np.array([[0,1]]); print("doing second part")
        for f in os.listdir(src_dir):
            #condition for number of created pictures
            if (n==limit*(i/2)): break
            #get name of imported background pic
            src_file = os.path.join(src_dir,f)
            #check_size = imageio.imread(src_file).astype(float)
            if f.endswith(file_ext):
                n+=1
                #Name of file
                new_name = ('{0}.png'.format(n))
                #Add some random function here for better data creation
                #Needs some label creation for half of data
                #if 1 make injection, if 2 just convert and rename
                if (i==1):
                    inject_photo(src_file, new_name, dst_dir)
                else:
                    copy_rename(src_file, new_name, dst_dir)
                #assign labels to array
                label[n-1] = mark
                if (n%(limit/10)==0):
                    print("done {}%".format((n/limit)*100))
        #save file with labels
        np.save("label.npy", label)
    print("Created {} files".format(n))

In [31]:
if __name__ == '__main__' :
    main()


creating 100 samples
done 10.0%
done 20.0%
done 30.0%
done 40.0%
done 50.0%
doing second part
done 60.0%
done 70.0%
done 80.0%
done 90.0%
done 100.0%
Created 100 files


In [56]:
check_name = np.load("label.npy")
print(check_name)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
